In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [3]:
earthquake_json['metadata']

{'generated': 1709278694000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-01-30&endtime=2024-02-29',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 11789}

In [4]:
type(earthquake_json['features'])

list

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.6,
  'place': '54 km S of Whites City, New Mexico',
  'time': 1709164025669,
  'updated': 1709165458952,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2024edug',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2024edug&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 39,
  'net': 'tx',
  'code': '2024edug',
  'ids': ',tx2024edug,',
  'sources': ',tx,',
  'types': ',origin,phase-data,',
  'nst': 15,
  'dmin': 0,
  'rms': 0.2,
  'gap': 58,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.6 - 54 km S of Whites City, New Mexico'},
 'geometry': {'type': 'Point', 'coordinates': [-104.421, 31.685, 6.0486]},
 'id': 'tx2024edug'}

### Convert to DataFrame
We need to grab the properties section out of every entry in the features JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.60,"54 km S of Whites City, New Mexico",1709164025669,1709165458952,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",tx2024edug,",",tx,",",origin,phase-data,",15.0,0.000000,0.20,58.0,ml,earthquake,"M 1.6 - 54 km S of Whites City, New Mexico"
1,1.85,"29 km E of Clayton, Idaho",1709163814990,1709223415150,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",mb90042268,",",mb,",",origin,phase-data,",8.0,0.137900,0.13,100.0,ml,earthquake,"M 1.9 - 29 km E of Clayton, Idaho"
2,1.67,"27 km E of Atka, Alaska",1709163203580,1709170925320,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91134478,",",av,",",origin,phase-data,",6.0,NaN,0.10,317.0,ml,earthquake,"M 1.7 - 27 km E of Atka, Alaska"
3,1.01,"14 km NE of Milford, Utah",1709162935420,1709164547240,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",uu80052841,",",uu,",",origin,phase-data,",11.0,0.008740,0.08,112.0,md,earthquake,"M 1.0 - 14 km NE of Milford, Utah"
4,1.23,"11 km E of Mammoth Lakes, CA",1709161614810,1709161707320,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc74009521,",",nc,",",nearby-cities,origin,phase-data,",9.0,0.004772,0.02,103.0,md,earthquake,"M 1.2 - 11 km E of Mammoth Lakes, CA"
